### CSIRO - Image2Biomass Prediction
URL: https://www.kaggle.com/competitions/csiro-biomass/overview

In [22]:
import torch
import torchvision
from torchvision.models import resnet18
from torchvision.models.feature_extraction import create_feature_extractor
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np

print(torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"

True


In [23]:
# data import
train_df = pd.read_csv('/kaggle/input/csiro-biomass/train.csv')
test_df = pd.read_csv('/kaggle/input/csiro-biomass/test.csv')
sample_submission = pd.read_csv('/kaggle/input/csiro-biomass/sample_submission.csv')
train_img = torch.utils.data.DataLoader('/kaggle/input/csiro-biomass/train')
train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)
train_df

,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,GDM_g,16.2750
...,...,...,...,...,...,...,...,...,...
1780,ID983582017__Dry_Clover_g,train/ID983582017.jpg,2015/9/1,WA,Ryegrass,0.64,9.0000,Dry_Clover_g,0.0000
1781,ID983582017__Dry_Dead_g,train/ID983582017.jpg,2015/9/1,WA,Ryegrass,0.64,9.0000,Dry_Dead_g,0.0000
1782,ID983582017__Dry_Green_g,train/ID983582017.jpg,2015/9/1,WA,Ryegrass,0.64,9.0000,Dry_Green_g,40.9400
1783,ID983582017__Dry_Total_g,train/ID983582017.jpg,2015/9/1,WA,Ryegrass,0.64,9.0000,Dry_Total_g,40.9400


In [ ]:
# extract target columns
y_train = torch.tensor(train_df['target'].values.astype(np.float32))
y_train = y_train.unsqueeze(1)

y_train

In [ ]:
# image
transform = transforms.Compose(
    [
        transforms.ToTenser(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)



In [ ]:
net = resnet18()
feature_extractor = create_feature_extractor(net, {"avgpool": "feature"})

feature_dict = feature_extractor(img)
feature_dict